In [8]:
import pandas as pd
import numpy as np
import re
import html
import time

# Importazioni scikit-learn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

# --- 1. CONFIGURAZIONE ---
TRAIN_FILE = 'winter_project_2026/development.csv'   
EVAL_FILE = 'winter_project_2026/evaluation.csv'     
OUTPUT_FILE = 'winter_project_2026/sample_submission.csv'

# Definizione della griglia di iperparametri da testare
# Il GridSearch proverà tutte le combinazioni per trovare la migliore F1 Macro
# PARAM_GRID = {
#     'tfidf__max_features': [10000, 20000],      # Vocabolario: 10k o 20k parole
#     'tfidf__ngram_range': [(1, 1), (1, 2)],     # Solo singole parole o anche coppie (bigrammi)
#     'clf__C': [0.1, 1, 10],                     # Regolarizzazione SVM (Minore = più generico)
# }

PARAM_GRID = {
    'preprocessor__tfidf__max_features': [50000, None],  # Proviamo ad aumentare ancora le parole
    'preprocessor__tfidf__ngram_range': [(1, 2)],         
    'clf__C': [0.1, 0.2, 0.5],  # Esploriamo intorno a 0.1
}

# --- 2. CARICAMENTO DATI ---
print(f"Caricamento dati...")
try:
    dev_df = pd.read_csv(TRAIN_FILE)
    eval_df = pd.read_csv(EVAL_FILE)
except FileNotFoundError:
    print(f"ERRORE: Assicurati che '{TRAIN_FILE}' e '{EVAL_FILE}' siano nella stessa cartella.")
    exit()

# --- 3. PREPROCESSING ---

# 1. Controllo Duplicati
duplicates = dev_df.duplicated(subset=['title', 'article']).sum()
print(f"Duplicati trovati (Titolo+Articolo): {duplicates}")

# 2. Controllo Lunghezza Testo
dev_df['text_len'] = (dev_df['title'].fillna('') + " " + dev_df['article'].fillna('')).apply(len)
print(f"Articoli con lunghezza 0 o < 50 chars: {(dev_df['text_len'] < 50).sum()}")

# 3. Importanza della Source
# print("\nEsempio: Top 3 Fonti per la classe 'Technology' (2):")
# tech_sources = dev_df[dev_df['label'] == 2]['source'].value_counts().head(3)
# print(tech_sources)
# # Vedrai che le fonti sono molto specifiche!
# print("-" * 30)

def preprocess_text(df, remove_duplicates=False):
    """
    Preprocessing avanzato che include la Fonte e gestisce i duplicati.
    """
    # 1. Rimozione duplicati (Solo per il training, mai per evaluation!)
    if remove_duplicates: 
        initial_len = len(df)
        # Rimuoviamo se titolo e articolo sono identici
        df = df.drop_duplicates(subset=['title', 'article'], keep='first').copy()
        print(f"Rimossi {initial_len - len(df)} duplicati.")
    else:
        # Anche se non rimuoviamo duplicati, facciamo una copia per sicurezza
        df = df.copy()

    # 2. Feature Engineering: Inseriamo la FONTE nel testo
    # La ripetiamo 3 volte per "urlarla" al modello (peso maggiore nel TF-IDF)
    source_feature = (df['source'].fillna('') + " ") * 3
    
    df['text_combined'] = source_feature + df['title'].fillna('') + " " + df['article'].fillna('')
    
    """ 
        VECCHIA FUNZIONE: AGGRESSIVA 
        Rimuove simboli utili (ad esempio, $, %, €), becessari per le classi Business e Technology.    
    """
    def clean(text):
        text = str(text).lower()
        text = html.unescape(text)
        text = re.sub(r'<[^>]+>', ' ', text)
        
        # --- MODIFICA QUI ---
        # Manteniamo lettere, numeri E simboli valuta/percentuale ($, €, %, £)
        # La regex precedente era: re.sub(r'[^a-z0-9\s]', ' ', text)
        text = re.sub(r'[^a-z0-9\s$€%£]', ' ', text) 
                
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    print("Pulizia e integrazione Source...")
    df['clean_text'] = df['text_combined'].apply(clean)
    return df

# --- AGGIUNTA FEATURE TEMPORALI ---
def extract_time_features(df):
    df = df.copy()
    # Convertiamo la stringa in oggetto datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    
    # Estraiamo l'ora (0-23) e il giorno della settimana (0=Lun, 6=Dom)
    df['hour'] = df['timestamp'].dt.hour.fillna(0)
    df['day_of_week'] = df['timestamp'].dt.dayofweek.fillna(0)
    return df
# 
# Applichiamo la trasformazione
print("Estrazione feature temporali...")
dev_df = extract_time_features(dev_df)
eval_df = extract_time_features(eval_df)

""" 
    B. Gestione dei Duplicati (Solo nel Training)
    Se nel development.csv  ci sono righe identiche, il modello va in overfitting su quelle frasi. 
    Dobbiamo rimuovere i duplicati dal set di training, ma MAI dal set di evaluation (perché dobbiamo predire per ogni ID richiesto).
"""
dev_df = preprocess_text(dev_df, remove_duplicates=True)
eval_df = preprocess_text(eval_df, remove_duplicates=False)

# Setup vettori
feature_cols = ['clean_text', 'page_rank', 'hour', 'day_of_week']
X_dev = dev_df[feature_cols]
X_eval = eval_df[feature_cols]

y_dev = dev_df['label'] 
eval_ids = eval_df['Id'] 

# --- 4. SPLIT TRAIN/VALIDATION ---
# Usiamo stratify per mantenere le proporzioni delle classi
X_train, X_val, y_train, y_val = train_test_split(
    X_dev, y_dev, test_size=0.2, random_state=42, stratify=y_dev
)

print(f"\nDati pronti. Train: {len(X_train)}, Val: {len(X_val)}")

# --- 5. PIPELINE E GRID SEARCH ---
# Definizione della struttura base
# --- AGGIORNAMENTO PIPELINE ---
# Usiamo ColumnTransformer per trattare diversamente testo e numeri
"""
Stiamo passando da un modello "solo testo" a un modello "ibrido". 
ColumnTransformer permette di dire al modello: "Usa TF-IDF sulla colonna di testo, ma usa la normalizzazione matematica sulla colonna numerica page_rank", 
e poi unisci tutto insieme prima di darlo in pasto al classificatore SVM. Senza StandardScaler, il valore del Page Rank (che può essere piccolo o grande) potrebbe essere ignorato o dominare ingiustamente rispetto ai valori TF-IDF.
"""
preprocessor = ColumnTransformer(
    transformers=[
        # Al testo applichiamo TF-IDF (nota: passiamo la colonna 'clean_text')
        ('tfidf', TfidfVectorizer(stop_words='english', min_df=3, sublinear_tf=True), 'clean_text'),
        
        # Al page_rank applichiamo uno scaler per normalizzarlo (media 0, var 1)
        ('num', StandardScaler(), ['page_rank', 'hour', 'day_of_week']) 
    ]
)

base_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('clf', LinearSVC(class_weight='balanced', random_state=42, dual='auto'))
])

print("\n--- Inizio Tuning Iperparametri (GridSearch) ---")
print(f"Combinazioni da testare: {len(PARAM_GRID['preprocessor__tfidf__max_features']) * len(PARAM_GRID['preprocessor__tfidf__ngram_range']) * len(PARAM_GRID['clf__C'])}")
start_time = time.time()

# Configurazione GridSearch
# scoring='f1_macro' è fondamentale come da specifiche 
grid = GridSearchCV(
    base_pipeline, 
    PARAM_GRID, 
    cv=3,                 # 3-Fold Cross Validation
    scoring='f1_macro',   # Metrica obiettivo
    n_jobs=-1,            # Usa tutti i processori
    verbose=1
)

grid.fit(X_train, y_train)

print(f"Tuning completato in {time.time() - start_time:.1f} secondi.")
print(f"Migliori parametri: {grid.best_params_}")
print(f"Miglior CV F1 Score: {grid.best_score_:.4f}")

# --- 6. VALIDAZIONE SUL SET DI VALIDATION ---
# Usiamo il modello migliore trovato per vedere come va sui dati mai visti (il 20%)
best_model_val = grid.best_estimator_
y_pred_val = best_model_val.predict(X_val)
val_f1 = f1_score(y_val, y_pred_val, average='macro')

print(f"\n>>> REPORT VALIDAZIONE (sul 20% hold-out) <<<")
print(f"Macro F1 Score reale: {val_f1:.4f}")
print(classification_report(y_val, y_pred_val))

# --- 7. ADDESTRAMENTO FINALE (RETRAINING) ---
# Ora che sappiamo quali parametri funzionano meglio, riaddestriamo 
# un modello nuovo su TUTTO il dataset di development (100% dei dati etichettati)
# per avere la massima conoscenza possibile prima della submission.

print("\n--- Riaddestramento Finale (Full Development Set) ---")
print("Applicazione dei migliori parametri all'intero dataset...")

final_pipeline = Pipeline([
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('tfidf', TfidfVectorizer(
                stop_words='english', 
                min_df=3,
                sublinear_tf=True,
                max_features=grid.best_params_['preprocessor__tfidf__max_features'],
                ngram_range=grid.best_params_['preprocessor__tfidf__ngram_range']
            ), 'clean_text'),
            ('num', StandardScaler(), ['page_rank', 'hour', 'day_of_week'])
        ]
    )),
    ('clf', LinearSVC(
        class_weight='balanced', 
        random_state=42, 
        dual='auto',
        C=grid.best_params_['clf__C']
    ))
])

final_pipeline.fit(X_dev, y_dev)
print("Modello finale pronto.")

# --- 8. GENERAZIONE FILE DI SUBMISSION ---
print(f"Predizione su {len(X_eval)} record di Evaluation...")
y_pred_eval = final_pipeline.predict(X_eval)

submission = pd.DataFrame({
    'Id': eval_ids,
    'Predicted': y_pred_eval
})

# Formattazione corretta CSV: Id, Predicted
submission.to_csv(OUTPUT_FILE, index=False)

print("\n" + "="*40)
print(f"COMPLETATO. File generato: {OUTPUT_FILE}")
print("="*40)
print("Verifica header:")
print(submission.head())

Caricamento dati...
Duplicati trovati (Titolo+Articolo): 2954
Articoli con lunghezza 0 o < 50 chars: 1524
Estrazione feature temporali...
Rimossi 2954 duplicati.
Pulizia e integrazione Source...
Pulizia e integrazione Source...

Dati pronti. Train: 61634, Val: 15409

--- Inizio Tuning Iperparametri (GridSearch) ---
Combinazioni da testare: 6
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Tuning completato in 171.2 secondi.
Migliori parametri: {'clf__C': 0.2, 'preprocessor__tfidf__max_features': None, 'preprocessor__tfidf__ngram_range': (1, 2)}
Miglior CV F1 Score: 0.7018

>>> REPORT VALIDAZIONE (sul 20% hold-out) <<<
Macro F1 Score reale: 0.7081
              precision    recall  f1-score   support

           0       0.75      0.74      0.74      4569
           1       0.74      0.82      0.78      2068
           2       0.81      0.83      0.82      2175
           3       0.64      0.53      0.58      1878
           4       0.81      0.95      0.87      1688
        